In [17]:
import os
import ast
import pandas as pd
import psycopg2
from collections import defaultdict
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# researchers_path = os.path.join(os.getcwd(), "datasets", "users", 'researchers.xlsx') 
# articles_path = os.path.join(os.getcwd(), "datasets", "articles", "excels_CS", "articles.xlsx")
# researchers_file_path = os.path.join(os.getcwd(), "datasets", "users", 'researchers.xlsx')

In [18]:
# connection details
db_user = 'postgres'
db_password = 'admin'
db_host = 'localhost'
db_port = '5432'
db_name = 'AOI'

# format special characters
password = quote_plus(db_password)

conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

# SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{password}@{db_host}:{db_port}/{db_name}')

# PostgreSQL tables -> pandas DataFrames
researchers_query = 'SELECT "Researcher ID", "Full Name", "Expertise", "Appreciated" FROM researchers_table'
articles_query = 'SELECT id, title, authors, url, subject_split FROM articles_table'

researchers_df = pd.read_sql(researchers_query, engine)
articles_df = pd.read_sql(articles_query, engine)

researchers_df.head()


,Researcher ID,Full Name,Expertise,Appreciated
0,AAG-9392-2021,"APOSTOL, Elena Simona","['Distributed systems', 'IT security', 'Parall...","arXiv:2310.02113, arXiv:2310.05269, arXiv:2310..."
1,JCE-1061-2023,"CARABAS, Costin","['Computer Science', 'Software Engineering', '...","arXiv:2310.00562, arXiv:2310.03736, arXiv:2310..."
2,L-6699-2016,"DEACONESCU, Razvan","['Systems', 'Security']","arXiv:2310.03598, arXiv:2310.03994, arXiv:2310..."
3,C-4537-2012,"SLUSANSCHI, Emil Ioan","['High performance computing', 'Computer syste...","arXiv:2310.02422, arXiv:2311.02840, arXiv:2310..."
4,GYU-2822-2022,"STANILOIU, Constantin Eduard","['Computer Science', 'Software Engineering']","arXiv:2310.00562, arXiv:2310.03736, arXiv:2310..."


In [19]:
# researchers_df = pd.read_excel(researchers_path, sheet_name="Sheet")
# articles_df = pd.read_excel(articles_path, sheet_name="Sheet1")

In [20]:
def assign_weights(interests):
    num_interests = len(interests)
  
    weights = []
    weight = 0.9

    for i in range(num_interests):
        weights.append((interests[i], round(weight, 3)))
        weight *= 0.8
    
    return weights

In [21]:
def compute_relevance(article_subjects, researcher_interests):
    researcher_interests_dict = dict(researcher_interests)
    matched_subjects = []
    
    # print("Researcher interests dict:", researcher_interests_dict)
    
    relevance_score = 0
    
    for subject in article_subjects:
        # print("Current article subj:", subject)
        for interest, weight in researcher_interests_dict.items():
            # print("Current researcher interest:", interest)
            if (interest.lower() == subject.lower()) or (subject.lower() == "security" and interest.lower() == "it security"):
                # print(f"match found: {interest} in {subject}")
                relevance_score += weight
                matched_subjects.append(interest)
    

    # print("Article Subjects:", article_subjects)
    # # print("Relevance Score:", relevance_score)
    # print()
    
    return relevance_score, matched_subjects


In [22]:
articles_recommended_to_researchers = defaultdict(set)
top_articles = defaultdict()
content_recom_filepath = os.path.join(os.getcwd(), "Content-Based_Recommendation_Results.md")

with open(content_recom_filepath, 'w', encoding='utf-8') as file:

    for index, researcher in researchers_df.iterrows():

        interests_as_string = researcher['Expertise']
        interests = ast.literal_eval(interests_as_string)
        num_interests = len(interests)

        weighted_interests = assign_weights(interests)
        # print(f"Weighted interests for {researcher['Full Name']}:", weighted_interests)

        articles_relevance = []

        for index_a, article in articles_df.iterrows():
            article_id = article['id']
            # print(f"Article id is {article_id}")
            article_subjects_as_string = article['subject_split']
            article_subjects = ast.literal_eval(article_subjects_as_string)
            # compute relevance score for current article
            relevance_score, matched_subjects = compute_relevance(article_subjects, weighted_interests)
            articles_relevance.append((article, relevance_score, matched_subjects))

        # Debugging..
        # print articles and their scores for  current researcher
        # print(f"\nArticles relevance for {researcher['Full Name']}:")
        
        # for article, relevance_score in articles_relevance:
        #     print(f"Article ID: {article['id']} | Title: {article['title']} | Relevance Score: {relevance_score}")
        #     print('*' * 60)

        articles_relevance.sort(key=lambda x: x[1], reverse=True)
        # select top 10 fits
        researcher_id = researcher['Researcher ID']
        top_articles[researcher_id] = articles_relevance[:10]

        for article, _, _ in top_articles[researcher_id]:
            article_id = article['id']
            articles_recommended_to_researchers[article_id].add(researcher['Full Name'])

        # select only common recomm
        articles_recommended_to_multiple_researchers = {article_id: researchers 
                                                        for article_id, researchers in articles_recommended_to_researchers.items() 
                                                        if len(researchers) >= 2}

        file.write("\n")
        file.write(f"# Researcher {researcher['Full Name']}\n")
        file.write(f"### **Expertise List:** {' | '.join(interests)}\n\n")
        file.write(f"### **Top 10 recommendations** according to interests for **{researcher['Full Name']}**:\n")
        file.write("| ID | URL | Title | Relevance Score | Matched Subjects |\n")
        file.write("| --- | --- | --- | --- | --- |\n")
        for article, relevance_score, matched_subjects in top_articles[researcher_id]:
            file.write(f"| {article['id']} | {article['url']} | {article['title']} | {relevance_score} | {matched_subjects} |\n")
        file.write("\n\n")


### Save common interests

In [23]:
# common interests (helper for later debugging?)
with open('articles_recommended_to_multiple_researchers.txt', 'w') as file:
    for article_id, researchers in articles_recommended_to_multiple_researchers.items():
        file.write(f"Article ID: {article_id}\n")
        file.write("Recommended to Researchers:\n")
        for researcher in researchers:
            file.write(f"- {researcher}\n")
        file.write("\n")

#### Update Appreciated column

In [24]:
# pd.set_option('display.max_colwidth', None)

researchers_df['Appreciated'] = researchers_df['Appreciated'].astype('object')

# for index, researcher in researchers_df.iterrows():
#     researcher_id = researcher['Researcher ID']
#     researcher_name = researcher['Full Name']
#     # ids of recommended articles
#     recommended_articles = [article['id'] for article, _, _ in top_articles[researcher_id] if researcher_name in articles_recommended_to_researchers[article['id']]]
#     recommended_articles_str = ', '.join(recommended_articles)
#     researchers_df.at[index, 'Appreciated'] = recommended_articles_str


# researchers_df.to_excel(researchers_file_path, index=False)

conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
# cursor object
cursor = conn.cursor()

# update the 'Appreciated' column in the 'researchers_table'
for index, researcher in researchers_df.iterrows():
    researcher_id = researcher['Researcher ID']
    researcher_name = researcher['Full Name']
    recommended_articles = [article['id'] for article, _, _ in top_articles[researcher_id] if researcher_name in articles_recommended_to_researchers[article['id']]]
    recommended_articles_str = ', '.join(recommended_articles)
    
    update_query = f"UPDATE researchers_table SET \"Appreciated\" = '{recommended_articles_str}' WHERE \"Researcher ID\" = '{researcher_id}'"
    cursor.execute(update_query)

conn.commit()

cursor.close()
conn.close()